<a href="https://colab.research.google.com/github/doanthanhnam1/DoanThanhNam/blob/main/final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from os import listdir
from numpy import asarray
from numpy import save
from keras import preprocessing
from tensorflow.keras.utils import load_img, img_to_array
import random
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator

In [2]:
data="/content/drive/MyDrive/cu_chi/data/"

In [3]:
pixels,labels=list(),list()

for folder in listdir(data):
    if folder!='.DS_Store':
        #print("Folder=",folder)
        for file in listdir(data  + folder):
            if file!='.DS_Store':
                #print("File=", file)
                pixel=load_img(data  + folder +"/" + file)
                pixel=img_to_array(pixel)
                pixels.append(pixel)
                labels.append( folder)

pixels =asarray(pixels)
labels =asarray(labels)#.reshape(-1,1)

In [4]:
from sklearn.preprocessing import LabelBinarizer
import pickle
from sklearn.model_selection import train_test_split
encoder = LabelBinarizer()
labels = encoder.fit_transform(labels)
#print(labels)

file = open('pix.data', 'wb')
# dump information to that file
pickle.dump((pixels,labels), file)
# close the file
file.close()

In [5]:
def load_data():
    file = open('pix.data', 'rb')
    # dump information to that file
    (pixels, labels) = pickle.load(file)
    # close the file
    file.close()
    return pixels, labels

#save_data()
x,y = load_data()
#random.shuffle(X)
x_train, x_test, y_train, y_test = train_test_split( x, y, test_size=0.2, random_state=100)
print(x_train.shape)
print(x_test.shape)

(771, 192, 128, 3)
(193, 192, 128, 3)


In [6]:
from keras.models import Sequential
from keras.layers import Dense

In [7]:
x_train_1= x_train.astype('float32')/255

x_test_1= x_test.astype('float32')/255

In [8]:
aug = ImageDataGenerator(rotation_range=20, zoom_range=0.1,
    rescale=1./255,
	width_shift_range=0.1,
    height_shift_range=0.1,
	horizontal_flip=True,
    brightness_range=[0.2,1.5], fill_mode="nearest")

aug_val = ImageDataGenerator(rescale=1./255)

In [9]:
from keras import models 
from keras.models import Sequential,Model
from keras import Input
from keras.layers import Dense,Flatten,Dropout,Conv2D,MaxPooling2D,Normalization
from keras.optimizers import Adam
classes=4

In [10]:
model=Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='linear',input_shape=(192,128,3),padding='same'))
model.add(MaxPooling2D((2,2),padding='same'))
model.add(Dropout(0.25))
#so lan tich chap
model.add(Conv2D(64,(3,3),activation='linear',padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((1,1),padding='same'))

In [11]:
model.add(Conv2D(128,(3,3),activation='linear',padding='same'))
model.add(Dropout(0.25))
model.add(MaxPooling2D((1,1),padding='same'))

In [12]:
from keras import optimizers
from keras import losses
model.add(Flatten())
model.add(Dense(128,activation='linear'))
model.add(Dropout(0.25))
model.add(Dense(1000,activation='linear'))
model.add(Dense(500,activation='linear'))
model.add(Dense(classes,activation='softmax'))
model.compile(loss=losses.categorical_crossentropy,optimizer=Adam(),metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 192, 128, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 96, 64, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 96, 64, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 64, 64)        18496     
                                                                 
 dropout_1 (Dropout)         (None, 96, 64, 64)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 96, 64, 64)       0         
 2D)                                                    

In [13]:
filepath="cu_chi_tay_project-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
train=model.fit_generator(aug.flow(x_train_1, y_train, batch_size=50),
                               epochs=100,
                               validation_data=aug.flow(x_test_1,y_test,
                               batch_size=50),
                               callbacks=callbacks_list)

<ipython-input-13-759345418e14>:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  train=model.fit_generator(aug.flow(x_train_1, y_train, batch_size=50),


Epoch 1/100
16/16 [==============================] - ETA: 0s - loss: 3.9237 - accuracy: 0.2451
Epoch 1: val_accuracy improved from -inf to 0.23316, saving model to cu_chi_tay_project-01-0.23.hdf5
16/16 [==============================] - 26s 906ms/step - loss: 3.9237 - accuracy: 0.2451 - val_loss: 1.7344 - val_accuracy: 0.2332
Epoch 2/100
16/16 [==============================] - ETA: 0s - loss: 1.7088 - accuracy: 0.2555
Epoch 2: val_accuracy improved from 0.23316 to 0.30052, saving model to cu_chi_tay_project-02-0.30.hdf5
16/16 [==============================] - 13s 835ms/step - loss: 1.7088 - accuracy: 0.2555 - val_loss: 1.4471 - val_accuracy: 0.3005
Epoch 3/100
16/16 [==============================] - ETA: 0s - loss: 1.4542 - accuracy: 0.2529
Epoch 3: val_accuracy did not improve from 0.30052
16/16 [==============================] - 8s 501ms/step - loss: 1.4542 - accuracy: 0.2529 - val_loss: 1.4086 - val_accuracy: 0.2124
Epoch 4/100
16/16 [==============================] - ETA: 0s - l